# SwellSight Real-to-Synthetic Pipeline - Setup and Installation (Enhanced)

This notebook handles environment preparation and dependency management for the SwellSight real-to-synthetic image generation pipeline with integrated utilities.

## New Features
- ✨ Integrated SwellSight utilities for better error handling and progress tracking
- 🔧 Automatic configuration management
- 📊 Memory optimization and monitoring
- 🛡️ Robust error handling with retry logic
- 📈 Progress tracking and performance feedback

---

## 1. Import SwellSight Utilities

In [6]:
import sys
import os
from pathlib import Path
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Add project root to Python path so `import utils` works
utils_path = Path.cwd()
if str(utils_path) not in sys.path:
    sys.path.insert(0, str(utils_path))

try:
    # Import SwellSight utilities
    from utils import (
        load_config, validate_config,
        validate_image_quality, validate_depth_map_quality,
        get_optimal_batch_size, cleanup_variables, monitor_memory,
        retry_with_backoff, handle_gpu_memory_error,
        create_progress_bar, display_stage_summary,
        save_stage_results, load_previous_results, check_dependencies
    )
    print("✅ SwellSight utilities loaded successfully")
except ImportError as e:
    print(f"⚠️ Could not import utilities: {e}")
    print("Continuing with basic functionality...")

⚠️ Could not import utilities: No module named 'utils'
Continuing with basic functionality...


## 2. Load Configuration and Environment Setup

In [2]:
# Load pipeline configuration
try:
    config = load_config("config.json")
    print(f"✅ Configuration loaded: {config['pipeline']['name']} v{config['pipeline']['version']}")
    
    # Extract commonly used settings
    batch_size = config['processing']['batch_size']
    quality_threshold = config['processing']['quality_threshold']
    data_dir = Path(config['paths']['data_dir'])
    output_dir = Path(config['paths']['output_dir'])
    checkpoint_dir = Path(config['paths']['checkpoint_dir'])
    
    print(f"📁 Data directory: {data_dir}")
    print(f"📁 Output directory: {output_dir}")
    print(f"📁 Checkpoint directory: {checkpoint_dir}")
    print(f"🎯 Quality threshold: {quality_threshold}")
    
except Exception as e:
    print(f"⚠️ Could not load configuration: {e}")
    print("Using default settings...")
    
    # Fallback configuration
    batch_size = "auto"
    quality_threshold = 0.7
    data_dir = Path("./data")
    output_dir = Path("./outputs")
    checkpoint_dir = Path("./checkpoints")

⚠️ Could not load configuration: name 'load_config' is not defined
Using default settings...


## 3. Environment Detection and Hardware Configuration

In [3]:
import torch
import platform

# Check if running in Google Colab
if 'google.colab' in sys.modules:
    from google.colab import drive
    print("🔗 Mounting Google Drive...")
    
    def mount_drive():
        drive.mount('/content/drive')
        return True
    
    try:
        # Use retry logic for drive mounting
        retry_with_backoff(mount_drive, max_retries=2)
        print("✅ Google Drive mounted successfully")
    except Exception as e:
        print(f"⚠️ Drive mounting failed: {e}")

# Hardware detection with memory monitoring
print("\n🔍 Hardware Detection:")
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Python: {sys.version.split()[0]}")

# GPU detection
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    current_gpu = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(current_gpu)
    gpu_memory = torch.cuda.get_device_properties(current_gpu).total_memory / (1024**3)
    
    print(f"🚀 GPU Available: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    print(f"🔢 GPU Count: {gpu_count}")
else:
    print("⚠️ No GPU available - using CPU")

# Memory monitoring
try:
    memory_info = monitor_memory()
    print(f"\n💻 System Memory: {memory_info.get('system_total_gb', 0):.1f} GB total, {memory_info.get('system_percent', 0):.1f}% used")
    if 'gpu_total_gb' in memory_info:
        print(f"🎮 GPU Memory: {memory_info.get('gpu_total_gb', 0):.1f} GB total, {memory_info.get('gpu_percent', 0):.1f}% used")
except Exception as e:
    print(f"⚠️ Could not get memory info: {e}")

🔗 Mounting Google Drive...
⚠️ Drive mounting failed: name 'retry_with_backoff' is not defined

🔍 Hardware Detection:
Platform: Linux 6.6.105+
Python: 3.12.12
🚀 GPU Available: NVIDIA A100-SXM4-40GB
💾 GPU Memory: 39.6 GB
🔢 GPU Count: 1
⚠️ Could not get memory info: name 'monitor_memory' is not defined


## 4. Create Directory Structure with Progress Tracking

In [4]:
# Create necessary directories
directories_to_create = [
    data_dir / "real" / "images",
    data_dir / "processed",
    data_dir / "depth_maps",
    data_dir / "synthetic",
    output_dir,
    checkpoint_dir,
    Path("models"),
    Path("logs")
]

print("📁 Creating directory structure...")
progress_bar = create_progress_bar(len(directories_to_create), "Creating directories")

created_dirs = []
for directory in directories_to_create:
    try:
        directory.mkdir(parents=True, exist_ok=True)
        created_dirs.append(str(directory))
        progress_bar.update(1)
    except Exception as e:
        print(f"⚠️ Could not create directory {directory}: {e}")

progress_bar.close()
print(f"✅ Created {len(created_dirs)} directories")

# Display directory structure
print("\n📂 Directory Structure:")
for directory in created_dirs:
    print(f"  📁 {directory}")

📁 Creating directory structure...


NameError: name 'create_progress_bar' is not defined

## 5. Install Dependencies with Error Handling

In [ ]:
import subprocess

# Define required packages
required_packages = [
    "torch>=1.9.0",
    "torchvision>=0.10.0",
    "transformers>=4.20.0",
    "diffusers>=0.10.0",
    "accelerate>=0.20.0",
    "opencv-python>=4.5.0",
    "Pillow>=8.0.0",
    "numpy>=1.21.0",
    "tqdm>=4.60.0",
    "psutil>=5.8.0"
]

def install_package(package):
    """Install a package using pip"""
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", package],
        capture_output=True,
        text=True
    )
    if result.returncode != 0:
        raise Exception(f"Installation failed: {result.stderr}")
    return result.stdout

print("📦 Installing required packages...")
progress_bar = create_progress_bar(len(required_packages), "Installing packages")

installed_packages = []
failed_packages = []

for package in required_packages:
    try:
        # Use retry logic for package installation
        retry_with_backoff(lambda: install_package(package), max_retries=2)
        installed_packages.append(package)
        progress_bar.update(1)
    except Exception as e:
        print(f"\n⚠️ Failed to install {package}: {e}")
        failed_packages.append(package)
        progress_bar.update(1)

progress_bar.close()

# Display installation summary
installation_metrics = {
    'total_packages': len(required_packages),
    'installed_successfully': len(installed_packages),
    'failed_installations': len(failed_packages),
    'success_rate': len(installed_packages) / len(required_packages)
}

display_stage_summary("Package Installation", installation_metrics)

if failed_packages:
    print("\n⚠️ Failed packages:")
    for package in failed_packages:
        print(f"  - {package}")
    print("\n💡 Try installing failed packages manually or check your internet connection.")

📦 Installing required packages...


Installing packages: 100%|███████████████████████████████████████| 10/10 [00:29<00:00,  2.91s/items]


STAGE SUMMARY: PACKAGE INSTALLATION
Total Packages: 10
Installed Successfully: 10
Failed Installations: 0
Success Rate: 1.000



## 6. Optimize Memory and Batch Size Configuration

In [ ]:
# Calculate optimal batch size based on available memory
print("🧠 Optimizing memory configuration...")

if batch_size == "auto":
    try:
        optimal_batch_size = get_optimal_batch_size(max_batch_size=32)
        print(f"✅ Calculated optimal batch size: {optimal_batch_size}")
    except Exception as e:
        print(f"⚠️ Could not calculate optimal batch size: {e}")
        optimal_batch_size = 4  # Conservative fallback
        print(f"Using conservative batch size: {optimal_batch_size}")
else:
    optimal_batch_size = batch_size
    print(f"✅ Using configured batch size: {optimal_batch_size}")

# Update configuration with optimized settings
try:
    config['processing']['batch_size'] = optimal_batch_size
    
    # Save updated configuration
    import json
    with open("config.json", "w") as f:
        json.dump(config, f, indent=2)
    
    print("✅ Configuration updated with optimized settings")
except Exception as e:
    print(f"⚠️ Could not save updated configuration: {e}")

# Display memory optimization suggestions
try:
    from utils.memory_optimizer import MemoryOptimizer
    optimizer = MemoryOptimizer()
    suggestions = optimizer.suggest_memory_optimizations()
    
    if suggestions:
        print("\n💡 Memory Optimization Suggestions:")
        for i, suggestion in enumerate(suggestions[:3], 1):
            print(f"  {i}. {suggestion}")
except Exception as e:
    print(f"⚠️ Could not generate optimization suggestions: {e}")

🧠 Optimizing memory configuration...
✅ Using configured batch size: 1
✅ Configuration updated with optimized settings

💡 Memory Optimization Suggestions:
  1. System memory usage is high (>80%). Consider reducing batch size.
  2. Close unnecessary applications to free up system memory.
  3. Process data in smaller batches to reduce memory pressure.


## 7. Save Setup Results and Environment Information

In [ ]:
# Collect environment information
environment_info = {
    'platform': platform.system(),
    'python_version': sys.version.split()[0],
    'torch_version': torch.__version__,
    'cuda_available': torch.cuda.is_available(),
    'gpu_count': torch.cuda.device_count() if torch.cuda.is_available() else 0,
    'optimal_batch_size': optimal_batch_size,
    'directories_created': created_dirs,
    'installed_packages': installed_packages,
    'failed_packages': failed_packages
}

if torch.cuda.is_available():
    environment_info.update({
        'gpu_name': torch.cuda.get_device_name(0),
        'gpu_memory_gb': torch.cuda.get_device_properties(0).total_memory / (1024**3)
    })

# Save setup results
setup_results = {
    'environment_info': environment_info,
    'configuration': config,
    'setup_status': 'completed',
    'installation_metrics': installation_metrics
}

setup_metadata = {
    'setup_time': '2024-01-12T00:00:00Z',  # This would be actual timestamp
    'notebook_version': '1.0_enhanced',
    'utilities_version': '1.0'
}

try:
    success = save_stage_results(setup_results, "setup", setup_metadata)
    if success:
        print("✅ Setup results saved successfully")
        print(f"📁 Results saved to: {output_dir / 'setup'}")
    else:
        print("⚠️ Could not save setup results")
except Exception as e:
    print(f"⚠️ Error saving setup results: {e}")

print("\n🎉 Setup and installation completed!")
print("\n📋 Next Steps:")
print("1. 📂 Add your real beach images to the data/real/images/ directory")
print("2. ▶️ Run notebook 02: Data Import and Preprocessing")
print("3. 🔄 Continue with the remaining pipeline notebooks")

## 8. Memory Cleanup

In [ ]:
# Clean up large variables to free memory
large_variables = [
    # Add any large variables that were created during setup
]

try:
    cleanup_variables(large_variables)
    print("✅ Memory cleanup completed")
    
    # Show final memory status
    final_memory = monitor_memory()
    print(f"💻 Final system memory usage: {final_memory.get('system_percent', 0):.1f}%")
    if 'gpu_percent' in final_memory:
        print(f"🎮 Final GPU memory usage: {final_memory.get('gpu_percent', 0):.1f}%")
        
except Exception as e:
    print(f"⚠️ Memory cleanup warning: {e}")